### <center>Main para pruebas y resultados</center>


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, RocCurveDisplay
from Data_Preproceso import DataPreproceso
from Modelo_XGB import ModeloXGBoost
from Voting_Classifier import VotingClassifierModel
from Stacking_Classifier import StackingClassifierModel

In [ ]:
data_preproceso = DataPreproceso('Data_fraude.csv')
data, preproceso = data_preproceso.cargar_data_y_preprocesar()

In [ ]:
modelo_xgb = ModeloXGBoost(data, preproceso)
modelo_xgb.crear_pipeline()
X_train, X_test, Y_train, Y_test = train_test_split(modelo_xgb.X, modelo_xgb.Y, test_size=0.25, random_state=7, stratify=modelo_xgb.Y)
modelo_xgb.pipeline.fit(X_train, Y_train)
Y_predict_proba_xgb = modelo_xgb.pipeline.predict_proba(X_test)[:, 1]
fpr_xgb, tpr_xgb, _ = roc_curve(Y_test, Y_predict_proba_xgb)
roc_auc_xgb = auc(fpr_xgb, tpr_xgb)

In [ ]:
voting_model = VotingClassifierModel(data, preproceso)
voting_model.crear_voting_classifier()
voting_model.pipeline.fit(X_train, Y_train)
Y_predict_proba_voting = voting_model.pipeline.predict_proba(X_test)[:, 1]
fpr_voting, tpr_voting, _ = roc_curve(Y_test, Y_predict_proba_voting)
roc_auc_voting = auc(fpr_voting, tpr_voting)

In [ ]:
stacking_model = StackingClassifierModel(data, preproceso)
stacking_model.crear_stacking_classifier()
stacking_model.pipeline.fit(X_train, Y_train)
Y_predict_proba_stacking = stacking_model.pipeline.predict_proba(X_test)[:, 1]
fpr_stacking, tpr_stacking, _ = roc_curve(Y_test, Y_predict_proba_stacking)
roc_auc_stacking = auc(fpr_stacking, tpr_stacking)

In [ ]:
plt.figure(figsize=(10, 8))
plt.plot(fpr_xgb, tpr_xgb, color='blue', lw=2, label=f'XGBoost ROC curve (area = {roc_auc_xgb:.2f})')
plt.plot(fpr_voting, tpr_voting, color='green', lw=2, label=f'VotingClassifier ROC curve (area = {roc_auc_voting:.2f})')
plt.plot(fpr_stacking, tpr_stacking, color='red', lw=2, label=f'StackingClassifier ROC curve (area = {roc_auc_stacking:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curves')
plt.legend(loc="lower right")
plt.show()